In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import torch

In [ ]:
df = pd.read_csv('to_kaggle.csv', encoding='utf-8')


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
possible_label = df.sentiment.unique()
dict_label = {}
for index,possible_label in enumerate(possible_label):
    dict_label[possible_label] = index
dict_label

{'Negative': 0, 'Positive': 1, 'Neutral': 2}

In [ ]:
df["Label"] = df["sentiment"].replace(dict_label)
df.head()

,Author_name,#Followers,Author FollowIndiadiadiag,Account Created,Verified,Location,Tweet,Length,Likes,Language,Retweets,Time,cleaned_tweets,sentiment,Label
0,barrie9reynolds,219,952,2018-01-31 21:42:28+00:00,False,"Toronto, Canada",RT @ClearBlueSee: NEW FOOTAGE - Russian Presid...,"[0, 140]",0,en,38,2022-04-03 15:27:50+00:00,new footage russian president putindiadiadia d...,Negative,0
1,AdvUmangShah,310,596,2013-10-28 16:37:38+00:00,False,"Gujarat, Bharat.",RT @rawnksood: I have loaded video on visit of...,"[0, 140]",0,en,11,2022-04-03 15:27:47+00:00,have loaded video on visit of russian foreign ...,Positive,1
2,partizan201415,2403,695,2014-05-29 10:05:44+00:00,False,Донецкая степь,RT @alyonational: Hello world. My name is Alyo...,"[0, 140]",0,en,2,2022-04-03 15:26:47+00:00,hello world my name is alyona ukraindiadiadiai...,Neutral,2
3,Aleksiukraine,84,138,2022-02-24 18:00:25+00:00,False,UkraIndiadiadiae,RT @alyonational: Hello world. My name is Alyo...,"[0, 140]",0,en,2,2022-04-03 15:26:14+00:00,hello world my name is alyona ukraindiadiadiai...,Neutral,2
4,JeffreyKleeven,31,57,2013-09-27 19:02:22+00:00,False,"Blerick-Centrum, Venlo","RT @anny_more: Hello world. My name is Ann, i'...","[0, 139]",0,en,3,2022-04-03 15:25:56+00:00,hello world my name is ann ukraindiadiadiaian ...,Neutral,2


In [ ]:
df['cleaned_tweets'].fillna(' ', inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_tweets'], df['Label'], test_size=0.2, random_state=42)

In [ ]:
# Convert pandas Series to list of strings
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()

# Encode train and test data
encoder_train = tokenizer.batch_encode_plus(X_train_list,
                                            add_special_tokens=True,
                                            return_attention_mask=True,
                                            padding='max_length',  # Use padding='max_length'
                                            max_length=256,
                                            return_tensors='pt')

encoder_test = tokenizer.batch_encode_plus(X_test_list,
                                           add_special_tokens=True,
                                           return_attention_mask=True,
                                           padding='max_length',  # Use padding='max_length'
                                           max_length=256,
                                           return_tensors='pt')


In [ ]:
input_ids_train = encoder_train['input_ids']
attention_masks_train = encoder_train["attention_mask"]
labels_train = torch.tensor(y_train.values)


input_ids_test = encoder_test['input_ids']
attention_masks_test = encoder_test["attention_mask"]
labels_test = torch.tensor(y_test.values)

In [ ]:
data_train = TensorDataset(input_ids_train,attention_masks_train,labels_train)
data_test = TensorDataset(input_ids_test,attention_masks_test,labels_test)

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                     num_labels = len(dict_label),
                                     output_attentions = False,
                                     output_hidden_states =  False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import RandomSampler,SequentialSampler,DataLoader

dataloader_train = DataLoader(
    data_train,
    sampler= RandomSampler(data_train),
    batch_size = 16

)


dataloader_test = DataLoader(
    data_test,
    sampler= RandomSampler(data_test),
    batch_size = 32

)

In [ ]:
from transformers import AdamW,get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)

epochs  = 10
scheduler = get_linear_schedule_with_warmup(
            optimizer,
    num_warmup_steps = 0,
   num_training_steps = len(dataloader_train)*epochs
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average = 'weighted')

In [ ]:
def accuracy_per_class(preds,labels):
    label_dict_reverse = {v:k for k,v in dict_label.items()}

    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f"Class:{label_dict_reverse}")
        print(f"Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n")

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"Loading:{device}")

Loading:cuda


In [ ]:
def evaluate(dataloader_val):
    model.eval()

    loss_val_total = 0
    predictions,true_vals = [],[]

    for batch in tqdm(dataloader_val):
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':  batch[0],
                  'attention_mask':batch[1],
                  'labels': batch[2]
                 }
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total +=loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0)
    return loss_val_avg,predictions,true_vals

In [ ]:
import torch
from tqdm.notebook import tqdm

In [ ]:

# for epoch in tqdm(range(1)):
#     print(epoch)
#     model.train()

#     loss_train_total=0

#     progress_bar = tqdm(dataloader_train,desc = "Epoch: {:1d}".format(epoch),leave = False,disable = False)


#     for batch in progress_bar:
#         model.zero_grad()

#         batch = tuple(b.to(device) for b in batch)

#         inputs = {
#             "input_ids":batch[0],
#             "attention_mask":batch[1],
#             "labels":batch[2]

#         }
#         outputs = model(**inputs)

#         loss = outputs[0]
# #         logits = outputs[1]
#         loss_train_total +=loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm(model.parameters(),1.0)

#         optimizer.step()
#         scheduler.step()


#         progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
#     torch.save(model.state_dict(),f'{epoch}.model')
#     tqdm.write('\nEpoch {epoch}')

#     loss_train_avg = loss_train_total/len(dataloader_train)
#     tqdm.write(f'Training Loss: {loss_train_avg}')
#     val_loss,predictions,true_vals = evaluate(dataloader_test)
#     test_score = f1_score_func(predictions,true_vals)
#     tqdm.write(f'Val Loss:{val_loss}\n Test Score:{test_score}')

  0%|          | 0/1 [00:00<?, ?it/s]

0


Epoch: 0:   0%|          | 0/1021 [00:00<?, ?it/s]

<ipython-input-23-e543ffd60384>:28: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(),1.0)



Epoch {epoch}
Training Loss: 0.43345078690402594


  0%|          | 0/128 [00:00<?, ?it/s]

Val Loss:0.23154118766251486
 Test Score:0.9261290194479832


In [ ]:
num_epochs = 5  # Set the number of epochs here

for epoch in tqdm(range(num_epochs)):
    print(f"Epoch: {epoch + 1}")
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc=f"Epoch: {epoch + 1}", leave=False, disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": batch[2]}

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    torch.save(model.state_dict(), f'{epoch + 1}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training Loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_test)
    test_score = f1_score_func(predictions, true_vals)

    tqdm.write(f'Val Loss:{val_loss}\nTest Score:{test_score}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1


Epoch: 1:   0%|          | 0/1021 [00:00<?, ?it/s]


Epoch {epoch}
Training Loss: 0.45517530626855446


  0%|          | 0/128 [00:00<?, ?it/s]

...

Val Loss:0.2855481870242329
Test Score:0.9491797513308634
Epoch: 5


Epoch: 5:   0%|          | 0/1021 [00:00<?, ?it/s]


Epoch {epoch}
Training Loss: 0.02648714516939512


  0%|          | 0/128 [00:00<?, ?it/s]

Val Loss:0.3175216905076468
Test Score:0.9533982585681225


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
val_loss, predictions, true_vals = evaluate(dataloader_test)

# Convert logits to predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(true_vals, predicted_labels, average='weighted')
recall = recall_score(true_vals, predicted_labels, average='weighted')
f1 = f1_score(true_vals, predicted_labels, average='weighted')

# Print the metrics
tqdm.write(f'Val Loss: {val_loss}')
tqdm.write(f'Precision: {precision}')
tqdm.write(f'Recall: {recall}')
tqdm.write(f'F1 Score: {f1}')


  0%|          | 0/128 [00:00<?, ?it/s]

Val Loss: 0.3175073851386969
Precision: 0.9535489004918773
Recall: 0.9534655890276758
F1 Score: 0.9533982585681225
